## Derivation of the Müllerian longitudinal axis in Visium samples 

In [ ]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt2
import seaborn as sns
import matplotlib as mpl
# import warnings
# warnings.filterwarnings("ignore")
import os
import skimage
from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial
import squidpy as sq
from PIL import Image
from PIL.TiffTags import TAGS
from skimage import io
import tifffile as tf
import scipy
print(skimage.__version__) # make sure version is 0.19.2
# from cellpose import plot
# from cellpose import models # my modfied code 
# from cellpose import io
#import cv2
import matplotlib.pyplot as plt

In [ ]:
path_to_reptract = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/'

In [ ]:
# Add metadata 
meta_dict = {'sample' : ['HCA_F_RepTsp13147665', 'HCA_F_RepTsp13147666', 'HCA_F_RepTsp13173760', 'HCA_F_RepTsp13173761', 
                       'HCA_F_RepTsp13173762', 'HCA_F_RepTsp13173763', 'HCA_F_RepTsp13219892', 'HCA_F_RepTsp13219893', 
                        'HCA_F_RepTsp13219894', 'HCA_F_RepTsp13219895', 'HCA_F_RepTsp13447718', 'HCA_F_RepTsp13447719',
                        'HCA_F_RepTsp13447720', 'HCA_F_RepTsp13447721',  'HCA_F_RepTsp13902013', 'HCA_F_RepTsp13902014', 'HCA_F_RepTsp13902015', 
                        'HCA_F_RepTsp13902016', 'HCA_F_RepTsp13902018', 'HCA_F_RepTsp14645816'],
            'sangerID' : ['Hrv58-GON-0-FO-1-s84', 'Hrv58-GON-0-FO-1-s80', 'HRV183-RPT-0-FO-1-S14_and_HRV183-RPT-0-FO-1-S12', 
                         'HRV183-RPT-0-FO-1-S9', 'HRV162-RPT-0-FO-1-S176_and_HRV183-RPT-0-FO-1-S26', 'HRV162-RPT-0-FO-1-S161_and_HRV162-RPT-0-FO-1-S162', 
                         'Hrv117-GON-0-FO-1-S109', 'HRV184-RPT-0-FO-2-S34_and_HRV184-RPT-0-FO-2-S31_and_Hrv117-GON-0-FO-1-S116', 
                         'HRV129-GON-0-FO-1-S42', 'HRV184-RPT-0-FO-1-S26', 'Hrv125-GON-0-FO-5-S65-A1', 
                         'Hrv125-GON-0-FO-5-S66-B1', 'Hrv125-GON-0-FO-5-S18-C1', 'Hrv125-GON-0-FO-5-S17-D1', 'Hrv238-RPT-0-FO-1-S39', 'Hrv238-RPT-0-FO-1-S24',
                          'Hrv214-RPT-0-FO-1-S58', 'Hrv214-RPT-0-FO-1-S57', 'Hrv245-RPT-9-FO-1-S13', 
                         'Hrv259-RPT-9-FO-1-S10/S11-i/ii'],
            'slide' : ['V12F14-072', 'V12F14-072', 'V12F14-073', 'V12F14-073', 'V12F14-073', 'V12F14-073', 
                      'V12F14-077', 'V12F14-077', 'V12F14-077', 'V12F14-077', 'V12Y31-055', 'V12Y31-055', 
                      'V12Y31-055', 'V12Y31-055', 'V42L11-100', 'V42L11-100', 'V42L18-060', 'V42L18-060', 
                      'V42L13-373', 'V42L06-066'], 
            'position' : ['A1', 'D1', 'A1', 'B1', 'C1', 'D1', 'A1', 'B1', 'C1', 'D1', 'A1', 'B1', 'C1', 'D1', 
                         'A1', 'D1', 'A1', 'D1', 'A1', 'A1'], 
#            'image_name' : ['V12F14-072_A1.tif', 'V12F14-072_D1.tif', 'V12F14-073_A1.tif', 'V12F14-073_B1.tif', 'V12F14-073_C1.tif', 'V12F14-073_D1.tif', 
#                           'V12F14-077_A1.tif', 'V12F14-077_B1.tif', 'V12F14-077_C1.tif', 'V12F14-077_D1.tif', 'V12Y31-055_A1.tiff',
#                           'V12Y31-055_B1.tiff', 'V12Y31-055_C1.tiff', 'V12Y31-055_D1.tiff'], 
            'donor' : ['Hrv58', 'Hrv58',  'Hrv183', 'Hrv183', 'Hrv162_and_Hrv183', 'Hrv162', 'Hrv117', 'Hrv184_and_Hrv117', 'Hrv129', 'Hrv184',
                      'Hrv125', 'Hrv125', 'Hrv125', 'Hrv125', 'Hrv238', 'Hrv238', 'Hrv214', 'Hrv214', 'Hrv245', 
                      'Hrv259'], 
            'stage(pcw)' : ['18', '18', '15', '15', '15_and_21', '21', '20', '15_and_20', '17', '15', 
                           '21','21','21','21', '17', '17', '15', '15', '15', '17'], 
            'sex' : ['female', 'female', 'female', 'female', 'female', 'female', 'male', 'male', 'male', 'male',
                    'female', 'female', 'female', 'female',  'female', 'female', 'female', 'female', 'female', 'female'], 
           'sectioning' : ['coronal', 'coronal', 'coronal', 'coronal', 'coronal_and_transverse', 'transverse', 'transverse', 'transverse_and_coronal', 
                          'transverse', 'transverse', 'coronal', 'coronal', 'sagittal', 'sagittal', 
                          'coronal', 
                          'coronal', 'sagittal', 'sagittal', 'sagittal', 'sagittal'], 
            'permeabilisation(min)' : [30, 30, 30, 30, 30, 30, 30, 30, 30, 30,  30, 30, 30, 30, 30, 30, 30, 30, 30, 30],
           'month_processing' : ['august_2022', 'august_2022', 'september_2022', 'september_2022', 'september_2022', 'september_2022', 
                                 'october_2022', 'october_2022', 'october_2022', 'october_2022', 
                                'january_2023', 'january_2023','january_2023','january_2023', 'july_2023', 'july_2023', 'july_2023', 'july_2023', 'july_2023',
                                'march_2024'], 
           'spaceranger' : ['2.0.0', '2.0.0', '2.0.0', '2.0.0', '2.0.0', '2.0.0', '2.0.0', '2.0.0', 
                            '2.0.0', '2.0.0', '2.0.0', '2.0.0', '2.0.0', '2.0.0', '2.1.0', '2.1.0', '2.1.0', '2.1.0', 
                           '2.1.0', '2.1.0']}
meta = pd.DataFrame.from_dict(meta_dict)
meta = meta.set_index('sample')
meta.head()

## 1. Import deconvoluted merged object with cell2location

In [ ]:
adata = sc.read('/lustre/scratch126/cellgen/team292/vl6/VISIUM/cell2location_v0.1_cytassist_visium/female_axis/predmodel/sp.h5ad')
adata

In [ ]:
adata.obs[adata.uns['mod']['factor_names']] = adata.obsm['q05_cell_abundance_w_sf']

In [ ]:
adata.obs['tot_cell_abundance'] = adata.uns["mod"]["post_sample_means"]["w_sf"].sum(1).flatten()
adata.obs['detection_sensit']  = adata.uns["mod"]["post_sample_q05"]["detection_y_s"]

In [ ]:
np.mean(adata.obs['tot_cell_abundance']>=5)

In [ ]:
adata.obs['broad_anatomy_annots'].value_counts(dropna = False)

In [ ]:
adata.obs['histology_annots'].value_counts(dropna = False)

In [ ]:
adata.obs['histology_annots'] = adata.obs['histology_annots'].astype(str)
adata = adata[[i not in ['nan'] for i in adata.obs['histology_annots']]]

In [ ]:
cell2loc_metadata = adata.copy()

In [ ]:
cell2loc_metadata

## 2. Import Visium Cytassist samples with annotated anatomy and histology

In [ ]:
sample_IDs = ['HCA_F_RepTsp13902013', 'HCA_F_RepTsp13902014', 'HCA_F_RepTsp13902015', 
                        'HCA_F_RepTsp13902016', 'HCA_F_RepTsp13902018', 'HCA_F_RepTsp14645816']

In [ ]:
path = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/'

In [ ]:
adatas = []
for sample in sample_IDs:
    print(sample)
    adata = sc.read(path+sample+'/'+sample+'_annotated.h5ad')
    adata.var_names_make_unique()
    adata.obs['sample'] = sample
    adata.var['SYMBOL'] = adata.var_names
    #adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    #adata.var_names = adata.var['ENSEMBL']
    #adata.var.drop(columns='ENSEMBL', inplace=True)

    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.var['mt'] = [gene.startswith('MT-') for gene in adata.var['SYMBOL']]
    adata.obs['mt_frac'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']

    # mitochondria-encoded (MT) genes should be removed for spatial mapping
    adata.obsm['mt'] = adata[:, adata.var['mt'].values].X.toarray()
    adata = adata[:, ~adata.var['mt'].values]

    # add sample name to obs names
    adata.obs["sample"] = [str(i) for i in adata.obs['sample']]
    adata.obs_names = adata.obs["sample"] \
                          + '_' + adata.obs_names
    adata.obs.index.name = 'spot_id'
    adata = adata[[i == 1.0 for i in adata.obs['in_tissue']]]
    print(adata.shape)
    adatas.append(adata)

In [ ]:
keys=[
        k
        for d in [
            adatas[0].uns["spatial"],
            adatas[1].uns["spatial"],
            adatas[2].uns["spatial"],
            adatas[3].uns["spatial"],
            adatas[4].uns["spatial"],
            adatas[5].uns["spatial"], 
        ]
        for k, v in d.items()]

In [ ]:
for adata in adatas:
    if 'x' in adata.obs.columns.to_list():
        del adata.obs['x']

In [ ]:
for adata in adatas:
    if 'y' in adata.obs.columns.to_list():
        del adata.obs['y']

In [ ]:
adatas_cytassist = adatas[0:6]

## 3. Concatenate into one object and add cell type deconvolution metadata

In [ ]:
adata_cytassist = sc.concat(
    adatas_cytassist,
    join = "outer",
    uns_merge="unique",
    keys=[
        k
        for d in [
            adatas_cytassist[0].uns["spatial"],
            adatas_cytassist[1].uns["spatial"],
            adatas_cytassist[2].uns["spatial"],
            adatas_cytassist[3].uns["spatial"],
            adatas_cytassist[4].uns["spatial"],
            adatas_cytassist[5].uns["spatial"],
        ]
        for k, v in d.items()
    ],
    index_unique=None,
)
adata_cytassist.var_names_make_unique()

In [ ]:
adata_cytassist.obs['sample'].value_counts()

In [ ]:
adata_cytassist.shape

In [ ]:
adata_cytassist.var.head()

In [ ]:
'TPT1' in adata_cytassist.var_names.to_list()

In [ ]:
'EEF1A1' in adata_cytassist.var_names.to_list()

In [ ]:
'HOXC8' in adata_cytassist.var_names.to_list()

In [ ]:
'HOXC8' in adata_cytassist.var_names.to_list()

In [ ]:
adata_cytassist

In [ ]:
for c in cell2loc_metadata.obs.columns.to_list():
    if c not in adata_cytassist.obs.columns.to_list():
        print(c)
        adata_cytassist.obs[c] = adata_cytassist.obs_names.map(cell2loc_metadata.obs[c].to_dict())

In [ ]:
adata_cytassist

## 4. Handy functions

### 4.1 Function to select slide from spatial anndata object

In [ ]:
def select_slide(adata, s, batch_key="sample"):
    r"""This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param batch_key: column in adata.obs listing experiment name for each location
    """

    slide = adata[adata.obs[batch_key].isin([s]), :].copy()
    s_keys = list(slide.uns["spatial"].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns["spatial"] = {s_spatial: slide.uns["spatial"][s_spatial]}

    return slide

### 4.2 Function to build the uterovaginal axis

In [ ]:
def transpose_axis(orig_axis, new_min, new_max):
    min_val = min(orig_axis)
    max_val = max(orig_axis)
    # Apply linear transformation
    dest_axis = [(x - min_val) / (max_val - min_val) * (new_max - new_min) + new_min for x in orig_axis]
    
    return dest_axis


In [ ]:
def build_uterovaginal_axis(adata, sample, min_cells, which_axis,
                            start_feature_segment1, end_feature_segment1, 
                            start_feature_segment2 = None, end_feature_segment2 = None, 
                            start_weight_segment1 = 1, end_weight_segment1 = 1,
                            start_weight_segment2 = 1, end_weight_segment2 = 1,
                            excluded_regions = ['UterusTransverse', 'Ovary', 'Epoophron', 'Unassigned', 'FallopianTube', 'LowerTract'], is_stitched = False, sample1 = None, sample2 = None):
    
    print('Is the sample stitched? {}'.format(is_stitched))
    
    if not is_stitched:
        
        # Select slide from spatial anndata object
        adata_sample = select_slide(adata, sample, batch_key = 'sample')
        
        # Remove excluded regions 
        adata_sample = adata_sample[[i not in excluded_regions for i in adata_sample.obs['broad_anatomy_annots']]]
        adata_sample = adata_sample[[i > min_cells for i in adata_sample.obs['tot_cell_abundance']]]
        
        if which_axis == 'uterocervical':
            print('Building uterocervical axis for sample {} from {} to {}'.format(sample, start_feature_segment1, 
                             end_feature_segment1))
        
            # Build uterocervical axis (first segment from uterine fundus to uterine bottom)
            df1 = adata_sample.obs[['L2_dist_longitudinal_anatomy_annots_' + start_feature_segment1, 'L2_dist_longitudinal_anatomy_annots_' + end_feature_segment1, 
               'broad_anatomy_annots', 'histology_annots']]
            print(df1.head())
            df1.dropna(inplace=True)
            uterovaginal_axis_segment1 = [( start_weight_segment1*int(s) - end_weight_segment1*int(e) ) / ( start_weight_segment1*int(s) + end_weight_segment1*int(e) ) for s,e in zip(df1['L2_dist_longitudinal_anatomy_annots_' + start_feature_segment1], df1['L2_dist_longitudinal_anatomy_annots_' + end_feature_segment1])]
        
            df1['UteroCervical_Axis'] = uterovaginal_axis_segment1
            print(df1['UteroCervical_Axis'].hist(bins=20))
        
            # Add uterocervical axis to slide 
            adata_sample.obs['UteroCervical_Axis'] = adata_sample.obs_names.map(df1['UteroCervical_Axis'].to_dict())
            sc.pl.spatial(adata_sample[[i in ['Epithelium', 'Mesenchyme', 
                                          'SmoothMuscle', 'Ligament'] for i in adata_sample.obs['histology_annots']]],
                                      color="UteroCervical_Axis", title = 'UteroCervical Axis', cmap = 'jet')
            
        elif which_axis == 'uterovaginal':
            print('Building uterovaginal axis for sample {} from {} to {}'.format(sample, start_feature_segment1, 
                            end_feature_segment2))
            
            # Build uterocervical axis (first segment from uterine fundus to uterine bottom)
            df1 = adata_sample.obs[['L2_dist_longitudinal_anatomy_annots_' + start_feature_segment1, 'L2_dist_longitudinal_anatomy_annots_' + end_feature_segment1, 
               'broad_anatomy_annots', 'histology_annots']]
            df1.dropna(inplace=True)
            uterovaginal_axis_segment1 = [( start_weight_segment1*int(s) - end_weight_segment1*int(e) ) / ( start_weight_segment1*int(s) + end_weight_segment1*int(e) ) for s,e in zip(df1['L2_dist_longitudinal_anatomy_annots_' + start_feature_segment1], df1['L2_dist_longitudinal_anatomy_annots_' + end_feature_segment1])]
        
            df1['UteroCervical_Axis'] = uterovaginal_axis_segment1
            print(df1['UteroCervical_Axis'].hist(bins=20))
        
            # Add uterocervical axis to slide 
            adata_sample.obs['UteroCervical_Axis'] = adata_sample.obs_names.map(df1['UteroCervical_Axis'].to_dict())
            sc.pl.spatial(adata_sample[[i in ['Epithelium', 'Mesenchyme', 
                                          'SmoothMuscle', 'Ligament'] for i in adata_sample.obs['histology_annots']]],
                                      color="UteroCervical_Axis", title = 'UteroCervical Axis', cmap = 'jet')

        
            # Build cervicovaginal axis (second segment from uterine fundus to lower tract)
            df2 = adata_sample.obs[['L2_dist_longitudinal_anatomy_annots_' + start_feature_segment2, 'L2_dist_longitudinal_anatomy_annots_' + end_feature_segment2, 
                   'broad_anatomy_annots', 'histology_annots']]
            df2.dropna(inplace=True)
            uterovaginal_axis_segment2 = [( start_weight_segment2*int(s) - end_weight_segment2*int(e) ) / ( start_weight_segment2*int(s) + end_weight_segment2*int(e) ) for s,e in zip(df2['L2_dist_longitudinal_anatomy_annots_' + start_feature_segment2], df2['L2_dist_longitudinal_anatomy_annots_' + end_feature_segment2])]

            df2['CervicoVaginal_Axis'] = uterovaginal_axis_segment2
            print(df2['CervicoVaginal_Axis'].hist(bins=20))

            # Add cervicovaginal axis to slide 
            adata_sample.obs['CervicoVaginal_Axis'] = adata_sample.obs_names.map(df2['CervicoVaginal_Axis'].to_dict())
            sc.pl.spatial(adata_sample[[i in ['Epithelium', 'Mesenchyme', 
                                              'SmoothMuscle', 'Ligament'] for i in adata_sample.obs['histology_annots']]],
                          color="CervicoVaginal_Axis", title = 'CervicoVaginal Axis', cmap = 'jet')
            
            # Build uterovaginal axis (combination of uterocervical and cervicovaginal)
            uterovaginal_axis_segment2_transpose = transpose_axis(df2['CervicoVaginal_Axis'], 1, 3)
            df2['CervicoVaginal_Axis_transpose'] = uterovaginal_axis_segment2_transpose
            print(df2['CervicoVaginal_Axis_transpose'].hist(bins=20))
            
            uterovaginal_axis = df1['UteroCervical_Axis'].append(df2['CervicoVaginal_Axis'])
            uterovaginal_axis['UteroVaginal_Axis'] = uterovaginal_axis['UteroCervical_Axis'].fillna(uterovaginal_axis['CervicoVaginal_Axis_transpose'])
        
            
        
        
        return adata_sample 
    
    else: 
        
        
        # Read stitched sample directly 
        adata_stitched = sc.read(path_to_reptract + sample1 + '_' + sample2 + '/' + sample1 + '_' + sample2 + '_annotated.h5ad')
        
        # Add a prefix to the distances computed on the joint object 
        for col in adata_stitched.obs.columns:
            if start_feature_segment2 != None:
                if col in [ 'L2_dist_longitudinal_anatomy_' + start_feature_segment1, 
                       'L2_dist_longitudinal_anatomy_' + end_feature_segment1, 
                       'L2_dist_longitudinal_anatomy_' + start_feature_segment2,
                       'L2_dist_longitudinal_anatomy_' + end_feature_segment2,
                       'broad_anatomy_annots', 'longitudinal_anatomy']:
                    adata_stitched.obs.rename(columns={col : 'joint_' + col}, inplace = True)
            else:
                if col in [ 'L2_dist_longitudinal_anatomy_' + start_feature_segment1, 
                       'L2_dist_longitudinal_anatomy_' + end_feature_segment1, 
                       'broad_anatomy_annots', 'longitudinal_anatomy']:
                    adata_stitched.obs.rename(columns={col : 'joint_' + col}, inplace = True)
                
        # Concatenate 
        adata_separate = select_slide(adata, sample1, batch_key = 'sample').concatenate(select_slide(adata, sample2, batch_key = 'sample'), index_unique = None)
        adata_separate = adata_separate[[i > min_cells for i in adata_separate.obs['tot_cell_abundance']]]
        
        # Import distances 
        for col in adata_stitched.obs.columns:
            if col.startswith('joint') or col == 'overlaps_with':
                adata_separate.obs[col] = adata_separate.obs_names.map(adata_stitched.obs[col].to_dict())
            
        # Create a dictionary mapping the index of the missing value to the index of the replacement value
        replacement_dict = adata_separate.obs['overlaps_with'].dropna().to_dict()
        replacement_dict = {v:k for k,v in replacement_dict.items()}
        
        to_remove = []
        for col in adata_separate.obs.columns:
            if col.startswith('joint'):
                for spot_id in adata_separate.obs_names:
                    value = adata_separate.obs.at[spot_id, col]
                    if isinstance(value, (float, np.float64)) and np.isnan(value):
                        if spot_id in list(replacement_dict.keys()):
                            overlapping_spot = replacement_dict[spot_id]
                            if overlapping_spot in adata_separate.obs_names:
                                adata_separate.obs.at[spot_id, col] = adata_separate.obs.at[overlapping_spot, col]
                        elif spot_id not in list(replacement_dict.keys()):
                            to_remove.append(spot_id)
        remove = list(np.unique(to_remove))
        
        # create a boolean mask of the observations to keep
        mask = ~adata_separate.obs_names.isin(remove)

        # select only the rows that match the mask
        adata_separate = adata_separate[mask] 
        
        adata_separate = adata_separate[[i not in excluded_regions for i in adata_separate.obs['broad_anatomy_annots']]]

        
        if which_axis == 'uterocervical':
            print('Building uterocervical axis for samples {} and {} from {} to {}'.format(sample1, sample2, start_feature_segment1, 
                             end_feature_segment1))
            

        
            # Build uterocervical axis (first segment from uterine fundus to uterine bottom)
            df1 = adata_separate.obs[['joint_L2_dist_longitudinal_anatomy_' + start_feature_segment1, 'joint_L2_dist_longitudinal_anatomy_' + end_feature_segment1, 
               'broad_anatomy_annots', 'histology_annots', 'joint_longitudinal_anatomy']]
            df1.dropna(inplace=True)
            uterovaginal_axis_segment1 = [( start_weight_segment1*int(s) - end_weight_segment1*int(e) ) / ( start_weight_segment1*int(s) + end_weight_segment1*int(e) ) for s,e in zip(df1['joint_L2_dist_longitudinal_anatomy_' + start_feature_segment1], df1['joint_L2_dist_longitudinal_anatomy_' + end_feature_segment1])]
        
            df1['UteroCervical_Axis'] = uterovaginal_axis_segment1
            print(df1['UteroCervical_Axis'].hist(bins=20, color = 'navy'))
        
            # Add uterocervical axis to slide 
            adata_separate.obs['UteroCervical_Axis'] = adata_separate.obs_names.map(df1['UteroCervical_Axis'].to_dict())
            
        elif which_axis == 'uterovaginal':
            print('Building uterovaginal axis for samples {} and {} from {} to {}'.format(sample1, sample2, start_feature_segment1, 
                            end_feature_segment2))
        
            # Build cervicovaginal axis (second segment from uterine fundus to lower tract)
            df2 = adata_separate.obs[['joint_L2_dist_longitudinal_anatomy_' + start_feature_segment1, 'joint_L2_dist_longitudinal_anatomy_' + end_feature_segment2, 
                   'broad_anatomy_annots', 'histology_annots', 'joint_longitudinal_anatomy']]
            print(df2.head())
            df2.dropna(inplace=True)
            uterovaginal_axis = [( start_weight_segment1*int(s) - end_weight_segment2*int(e) ) / ( start_weight_segment1*int(s) + end_weight_segment2*int(e) ) for s,e in zip(df2['joint_L2_dist_longitudinal_anatomy_' + start_feature_segment1], df2['joint_L2_dist_longitudinal_anatomy_' + end_feature_segment2])]

            df2['UteroVaginal_Axis'] = uterovaginal_axis
            plt.figure(figsize=(10, 5))  # Set the figure size
            plt.subplot(1, 2, 1)
            plt.hist(df2['UteroVaginal_Axis'], bins=20, alpha=0.7, color='blue', label='UteroVaginal_Axis')
            adata_separate.obs['UteroVaginal_Axis'] = adata_separate.obs_names.map(df2['UteroVaginal_Axis'].to_dict())

            # Recode uterovaginal axis so that 'UterineBottom' is found at position +1
            # Find the mean coordinate along the uterovaginal axis where the longitudinal anatomical annotation is uterine bottom 
            mean_coordinate_uterinebottom = df2.loc[df2['joint_longitudinal_anatomy'] == 'UterineBottom', 'UteroVaginal_Axis'].mean()
            print(mean_coordinate_uterinebottom)
            
            b = (1-mean_coordinate_uterinebottom)/(1+mean_coordinate_uterinebottom)
            a = b+1
            df2['UteroVaginal_Axis_Recoded'] = a * df2['UteroVaginal_Axis'] + b
            plt.subplot(1, 2, 2)
            plt.hist(df2['UteroVaginal_Axis_Recoded'], bins=20, alpha=0.7, color='blue', label='UteroVaginal_Axis_Recoded')
            plt.tight_layout()  # Adjust layout to prevent overlapping
            plt.show()
            # Add cervicovaginal axis to slide 
            adata_separate.obs['UteroVaginal_Axis_Recoded'] = adata_separate.obs_names.map(df2['UteroVaginal_Axis_Recoded'].to_dict())
            df2['UteroVaginal_Axis_forPlotting'] = transpose_axis(df2['UteroVaginal_Axis_Recoded'], 0, 4)
            plt.hist(df2['UteroVaginal_Axis_forPlotting'], bins=20, alpha=0.7, color='blue', label='UteroVaginal_Axis_forPlotting')
            plt.tight_layout()  # Adjust layout to prevent overlapping
            plt.show()
            adata_separate.obs['UteroVaginal_Axis_forPlotting'] = adata_separate.obs_names.map(df2['UteroVaginal_Axis_forPlotting'].to_dict())
             

        return adata_separate.obs
        


In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, 
                         fontsize=14, figsize=(10, 10), color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
adata_cytassist.obs['broad_anatomy_annots'].value_counts(dropna = False)

In [ ]:
adata_cytassist

In [ ]:
adata_cytassist.obs['longitudinal_anatomy'].value_counts(dropna = False)

## 5. Compute uterovaginal axis

In [ ]:
sample1 = 'HCA_F_RepTsp13902013'
sample2 = 'HCA_F_RepTsp13902014'

In [ ]:
HCA_F_RepTsp13902013_HCA_F_RepTsp13902014_axis = build_uterovaginal_axis(adata_cytassist, None, 10, 'uterovaginal',
                    'UterineFundus', 'UterineBottom', start_feature_segment2 = 'UterineBottom', end_feature_segment2 = 'EndodermalVaginalStart', start_weight_segment1 = 0.5, end_weight_segment1 = 0.5,
                     start_weight_segment2 = 0.5, end_weight_segment2 = 0.5, is_stitched = True,
                       sample1 = 'HCA_F_RepTsp13902013', sample2 = 'HCA_F_RepTsp13902014', 
                        excluded_regions = ['UterusTransverse', 'VaginalPlate', 'Ovary', 'Epoophron', 'Unassigned', 'FallopianTube', 'ExternalGenitalia', 'LowerTract'])


In [ ]:
HCA_F_RepTsp13902013_HCA_F_RepTsp13902014_axis

In [ ]:
HCA_F_RepTsp13902013 = select_slide(adata_cytassist, 'HCA_F_RepTsp13902013', batch_key  = 'sample')
HCA_F_RepTsp13902013.obs['UteroVaginal_Axis_Recoded'] = HCA_F_RepTsp13902013.obs_names.map(HCA_F_RepTsp13902013_HCA_F_RepTsp13902014_axis['UteroVaginal_Axis_Recoded'].to_dict())
HCA_F_RepTsp13902013.obs['UteroVaginal_Axis_forPlotting'] = HCA_F_RepTsp13902013.obs_names.map(HCA_F_RepTsp13902013_HCA_F_RepTsp13902014_axis['UteroVaginal_Axis_forPlotting'].to_dict())

sc.pl.spatial(HCA_F_RepTsp13902013, color="UteroVaginal_Axis_Recoded", cmap = 'jet')
sc.pl.spatial(HCA_F_RepTsp13902013, color="UteroVaginal_Axis_forPlotting", cmap = 'jet')

In [ ]:
HCA_F_RepTsp13902013

In [ ]:
HCA_F_RepTsp13902014 = select_slide(adata_cytassist, 'HCA_F_RepTsp13902014', batch_key  = 'sample')
HCA_F_RepTsp13902014.obs['UteroVaginal_Axis_Recoded'] = HCA_F_RepTsp13902014.obs_names.map(HCA_F_RepTsp13902013_HCA_F_RepTsp13902014_axis['UteroVaginal_Axis_Recoded'].to_dict())
HCA_F_RepTsp13902014.obs['UteroVaginal_Axis_forPlotting'] = HCA_F_RepTsp13902014.obs_names.map(HCA_F_RepTsp13902013_HCA_F_RepTsp13902014_axis['UteroVaginal_Axis_forPlotting'].to_dict())

sc.pl.spatial(HCA_F_RepTsp13902014, color="UteroVaginal_Axis_Recoded", cmap = 'jet')
sc.pl.spatial(HCA_F_RepTsp13902014, color="UteroVaginal_Axis_forPlotting", cmap = 'jet')

In [ ]:
HCA_F_RepTsp13902013_HCA_F_RepTsp13902014_axis.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/HCA_F_RepTsp13902013_HCA_F_RepTsp13902014/UterinoVaginalAxis_Mullerian.csv')

In [ ]:
HCA_F_RepTsp13902015_HCA_F_RepTsp13902016_axis = build_uterovaginal_axis(adata_cytassist, None, 10, 'uterovaginal',
                    'UterineFundus', 'UterineBottom', start_feature_segment2 = 'UterineBottom', end_feature_segment2 = 'EndodermalVaginalStart', start_weight_segment1 = 0.5, end_weight_segment1 = 0.5,
                     start_weight_segment2 = 0.5, end_weight_segment2 = 0.5, is_stitched = True,
                       sample1 = 'HCA_F_RepTsp13902015', sample2 = 'HCA_F_RepTsp13902016', 
                        excluded_regions = ['UterusTransverse', 'VaginalPlate', 'Ovary', 'Epoophron', 'Unassigned', 'FallopianTube', 
                                           'LowerTract', 'ExternalGenitalia'])


In [ ]:
HCA_F_RepTsp13902015 = select_slide(adata_cytassist, 'HCA_F_RepTsp13902015', batch_key  = 'sample')
HCA_F_RepTsp13902015.obs['UteroVaginal_Axis_Recoded'] = HCA_F_RepTsp13902015.obs_names.map(HCA_F_RepTsp13902015_HCA_F_RepTsp13902016_axis['UteroVaginal_Axis_Recoded'].to_dict())
HCA_F_RepTsp13902015.obs['UteroVaginal_Axis_forPlotting'] = HCA_F_RepTsp13902015.obs_names.map(HCA_F_RepTsp13902015_HCA_F_RepTsp13902016_axis['UteroVaginal_Axis_forPlotting'].to_dict())

sc.pl.spatial(HCA_F_RepTsp13902015, color="UteroVaginal_Axis_Recoded", cmap = 'jet')
sc.pl.spatial(HCA_F_RepTsp13902015, color="UteroVaginal_Axis_forPlotting", cmap = 'jet')

In [ ]:
HCA_F_RepTsp13902016 = select_slide(adata_cytassist, 'HCA_F_RepTsp13902016', batch_key  = 'sample')
HCA_F_RepTsp13902016.obs['UteroVaginal_Axis_Recoded'] = HCA_F_RepTsp13902016.obs_names.map(HCA_F_RepTsp13902015_HCA_F_RepTsp13902016_axis['UteroVaginal_Axis_Recoded'].to_dict())
HCA_F_RepTsp13902016.obs['UteroVaginal_Axis_forPlotting'] = HCA_F_RepTsp13902016.obs_names.map(HCA_F_RepTsp13902015_HCA_F_RepTsp13902016_axis['UteroVaginal_Axis_forPlotting'].to_dict())

sc.pl.spatial(HCA_F_RepTsp13902016, color="UteroVaginal_Axis_Recoded", cmap = 'jet')
sc.pl.spatial(HCA_F_RepTsp13902016, color="UteroVaginal_Axis_forPlotting", cmap = 'jet')

In [ ]:
HCA_F_RepTsp13902015_HCA_F_RepTsp13902016_axis.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/HCA_F_RepTsp13902015_HCA_F_RepTsp13902016/UteroVaginalAxis_Mullerian.csv')


## 6. Combine the uterovaginal axis from multiple samples 

In [ ]:
def combine_axis(samples, axis_name, return_adata = False):
    
    # Merge anndata objects
    adata_all = samples[0].concatenate(samples[1:], index_unique = None)
    
    # Remove spots that don't have a value for the axis 
    adata_all_obs = adata_all.obs
    adata_all_obs = adata_all_obs[adata_all_obs[axis_name].notna()]
    adata_all = adata_all[adata_all_obs.index.to_list(), :]
    
    if return_adata:
        return adata_all
    else:
        return adata_all_obs

In [ ]:
uterovaginal_axis = combine_axis(samples = [ 
                                            HCA_F_RepTsp13902013, HCA_F_RepTsp13902014, HCA_F_RepTsp13902015, HCA_F_RepTsp13902016
                                           ], 
                                axis_name = 'UteroVaginal_Axis_forPlotting', return_adata = True)

In [ ]:
uterovaginal_axis.shape

In [ ]:
uterovaginal_axis.var.head()

In [ ]:
uterovaginal_axis.obs['histology_annots'].value_counts(dropna = False)

In [ ]:
uterovaginal_axis.obs['broad_anatomy_annots'].value_counts(dropna = False)

In [ ]:
uterovaginal_axis.obs.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/uterovaginal_axis_mullerian.csv')

In [ ]:
uterovaginal_axis

In [ ]:
# Selecting relevant columns for computation
relevant_columns = ['Coelomic_Epithelium', 'FallopianTube_Epithelium', 'Ciliated_Epithelium', 'FallopianTube_Mesenchyme', 'FallopianTube_Ligament', 'FallopianTube_SmoothMuscle', 'Uterus/Cervix_Epithelium', 'Uterus_Mesenchyme', 'Uterus_Ligament', 'Uterus_SmoothMuscle', 'Cervix_Mesenchyme', 'Cervix/MüllerianVagina_Ligament', 'Cervix/MüllerianVagina_SmoothMuscle', 'MüllerianVagina_Epithelium', 'MüllerianVagina_Mesenchyme', 'VaginalPlate_Epithelium', 'VaginalPlate_Mesenchyme', 'VaginalPlate_SmoothMuscle', 'VaginalPlate_Ligament', 'MesonephricTubules_Epithelium', 'Epoophron_Mesenchyme', 'Pre-Perivascular', 'Perivascular', 'SkeletalMuscle', 'Endothelial', 'Endothelial_Lymphatic', 'Immune', 'Erythroid', 'SchwannCell', 'Neural']

# Calculating the percentage of cells that are of the most abundant cell type for each spot
uterovaginal_axis.obs['Most_Abundant_Cell_Type'] = uterovaginal_axis.obs[relevant_columns].idxmax(axis=1)


In [ ]:
uterovaginal_axis.obs['Most_Abundant_Cell_Type'].value_counts()

In [ ]:
uterovaginal_axis.obs[relevant_columns].sum(axis=1).values

In [ ]:
for s in uterovaginal_axis.obs_names:
    ct = uterovaginal_axis.obs.loc[s, 'Most_Abundant_Cell_Type']
    n_ct = uterovaginal_axis.obs.loc[s, ct]
    n_tot = uterovaginal_axis.obs.loc[s, relevant_columns].sum()
    uterovaginal_axis.obs.loc[s, 'Percentage_Most_Abundant_Cell_Type'] = (n_ct / n_tot) 

In [ ]:
uterovaginal_axis.obs['Percentage_Most_Abundant_Cell_Type']

## 7. Uterovaginal axis in mesenchymal spots -- prepare for TradeSeq

In [ ]:
uterus_mese = uterovaginal_axis[[i in ['Uterus_Mesenchyme', 'Cervix_Mesenchyme',
                'MüllerianVagina_Mesenchyme'] for i in uterovaginal_axis.obs['Most_Abundant_Cell_Type']]]
uterus_mese

In [ ]:
uterus_mese.obs['Percentage_Most_Abundant_Cell_Type'].hist(bins=20)

In [ ]:
uterus_mese = uterus_mese[[i > 0.6 for i in uterus_mese.obs['Percentage_Most_Abundant_Cell_Type']]]

In [ ]:

HCA_F_RepTsp13902013.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902013.obs_names.map(uterus_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902013, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902013.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902013.obs_names.map(uterus_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902013, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp13902014.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902014.obs_names.map(uterus_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902014, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902014.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902014.obs_names.map(uterus_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902014, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp13902015.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902015.obs_names.map(uterus_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902015, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902015.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902015.obs_names.map(uterus_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902015, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp13902016.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902016.obs_names.map(uterus_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902016, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902016.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902016.obs_names.map(uterus_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902016, color="Most_Abundant_Cell_Type")



In [ ]:
import anndata

In [ ]:
uterus_mese.X.toarray()

In [ ]:
uterus_mese_forR = uterus_mese.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in uterus_mese_forR.var_names if not name.startswith('MT-')]
uterus_mese_forR = uterus_mese_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in uterus_mese_forR.var_names if not name.startswith('RP')]
uterus_mese_forR = uterus_mese_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in uterus_mese_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
uterus_mese_forR = uterus_mese_forR[:, non_haemo_genes]

sc.pp.filter_genes(uterus_mese_forR, min_cells=10)
sc.pp.filter_genes(uterus_mese_forR, min_counts=1)


In [ ]:
'HLA-B' in uterus_mese_forR.var_names

In [ ]:
uterus_mese_forR.shape

In [ ]:
uterus_mese_forR.obs['UteroVaginal_Axis_forPlotting'].hist(bins = 10)

In [ ]:
for col in uterus_mese_forR.obs.columns:
    if col not in [ 'UteroVaginal_Axis_forPlotting', 'sample']:
        del uterus_mese_forR.obs[col]

In [ ]:
del uterus_mese_forR.obsm

In [ ]:
uterus_mese_forR.obs.head()

In [ ]:
uterus_mese_forR.obs.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/UterovaginalAxis_mese_spots_mullerian_uterovaginal_cytassist.csv')


In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R -i uterus_mese_forR
uterus_mese_forR

In [ ]:
%%R 
saveRDS(uterus_mese_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/UterovaginalAxis_mese_mullerian_uterovaginal_cytassist.rds')

In [ ]:
uterovaginal_axis.obs['Most_Abundant_Cell_Type'].value_counts()

## 8. Uterovaginal axis in epithelial spots -- prepare for TradeSeq

In [ ]:
# uterus_epi = uterovaginal_axis[[i in ['Epithelial_UCA1_Uterus', 
#                                       'Epithelial_UCA1+TP63'] for i in uterovaginal_axis.obs['Most_Abundant_Cell_Type']]]
# uterus_epi

In [ ]:
uterus_epi = uterovaginal_axis[[i in ['Epithelium'] for i in uterovaginal_axis.obs['histology_annots']]]
uterus_epi

In [ ]:
uterus_epi.obs['Percentage_Most_Abundant_Cell_Type'].hist(bins=50)

In [ ]:
uterus_epi.obs['Most_Abundant_Cell_Type'].value_counts()

In [ ]:
#uterus_epi = uterus_epi[[i > 0.5 for i in uterus_epi.obs['Percentage_Most_Abundant_Cell_Type']]]

In [ ]:
uterus_epi.shape

In [ ]:
uterus_epi.shape

In [ ]:


HCA_F_RepTsp13902013.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902013.obs_names.map(uterus_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902013, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902013.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902013.obs_names.map(uterus_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902013, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp13902014.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902014.obs_names.map(uterus_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902014, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902014.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902014.obs_names.map(uterus_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902014, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp13902015.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902015.obs_names.map(uterus_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902015, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902015.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902015.obs_names.map(uterus_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902015, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp13902016.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902016.obs_names.map(uterus_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902016, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902016.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902016.obs_names.map(uterus_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902016, color="Most_Abundant_Cell_Type")


In [ ]:
uterus_epi.shape

In [ ]:
uterus_epi_forR = uterus_epi.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in uterus_epi_forR.var_names if not name.startswith('MT-')]
uterus_epi_forR = uterus_epi_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in uterus_epi_forR.var_names if not name.startswith('RP')]
uterus_epi_forR = uterus_epi_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in uterus_epi_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
uterus_epi_forR = uterus_epi_forR[:, non_haemo_genes]

sc.pp.filter_genes(uterus_epi_forR, min_cells = 10)
sc.pp.filter_genes(uterus_epi_forR, min_counts = 1)

In [ ]:
for col in uterus_epi_forR.obs.columns:
    if col not in [ 'UteroVaginal_Axis_forPlotting', 'sample']:
        del uterus_epi_forR.obs[col]

In [ ]:
del uterus_epi_forR.obsm

In [ ]:
uterus_epi_forR.obs.head()

In [ ]:
uterus_epi_forR.obs['UteroVaginal_Axis_forPlotting'].hist(bins =10)

In [ ]:
uterus_epi_forR.obs.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/UterovaginalAxis_epi_spots_mullerian_uterovaginal_cytassist.csv')


In [ ]:
%%R -i uterus_epi_forR
uterus_epi_forR

In [ ]:
%%R 
saveRDS(uterus_epi_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/UterovaginalAxis_epi_mullerian_uterovaginal_cytassist.rds')

In [ ]:
uterus_epi_forR.shape

In [ ]:
uterovaginal_axis.shape

In [ ]:
uterovaginal_axis = uterovaginal_axis.obs

## 9. Fallopian tube axis 

In [ ]:
fallopian_HCA_F_RepTsp13902018 = sc.read('/lustre/scratch126/cellgen/team292/vl6/VISIUM/HCA_F_RepTsp13902018/HCA_F_RepTsp13902018_annotated.h5ad')
fallopian_HCA_F_RepTsp13902018

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, 
                         fontsize=14, figsize=(7, 7), color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
sc.pl.spatial(fallopian_HCA_F_RepTsp13902018, save = '_15pcw_FT')

In [ ]:
def build_fallopian_axis(adata, sample, min_cells, start_feature, end_feature, start_weight = 1, end_weight = 1,
                         excluded_regions = ['TransverseUterus', 'VaginalPlate', 'Unassigned', 'Uterus', 'Ovary', 
                                            'Epoophron', 'Background'], is_stitched = False, sample1 = None, sample2 = None):
    
    print('Is the sample stitched? {}'.format(is_stitched))
    
    if not is_stitched:
        
        print('Building uterovaginal axis for sample {} from {} (weight: {}) to {} (weight: {})'.format(sample, start_feature, 
                            start_weight, end_feature, end_weight))
        
        # Select slide from spatial anndata object
        adata_sample = select_slide(adata, sample, batch_key = 'sample')
        
        # Remove excluded regions 
        adata_sample = adata_sample[[i not in excluded_regions for i in adata_sample.obs['broad_anatomy_annots']]]
        adata_sample = adata_sample[[i > min_cells for i in adata_sample.obs['tot_cell_abundance']]]
        adata_sample = adata_sample[[i != 'Background' for i in adata_sample.obs['longitudinal_anatomy']]]
    
        adata_sample = adata_sample[[notna_val for notna_val in adata_sample.obs['L2_dist_longitudinal_anatomy_FallopianFimbriae'].notna()]]
        
        # Build axis 
        df = adata_sample.obs[['L2_dist_longitudinal_anatomy_' + start_feature, 'L2_dist_longitudinal_anatomy_' + end_feature, 
               'broad_anatomy_annots', 'histology_annots']]
        print(df.head())
        print(df['L2_dist_longitudinal_anatomy_' + start_feature].isna().value_counts())
        fallopian_axis = [( start_weight*int(s) - end_weight*int(e) ) / ( start_weight*int(s) + end_weight*int(e) ) for s,e in zip(df['L2_dist_longitudinal_anatomy_' + start_feature], df['L2_dist_longitudinal_anatomy_' + end_feature])]
        df['Fallopian_Axis'] = fallopian_axis
        df['Fallopian_Axis'] = transpose_axis(df['Fallopian_Axis'], -2, 0)
        print(df['Fallopian_Axis'].hist(bins=20))
        
        # Add axis to slide 
        adata_sample.obs['Fallopian_Axis'] = adata_sample.obs_names.map(df['Fallopian_Axis'].to_dict())
        sc.pl.spatial(adata_sample, color="Fallopian_Axis", cmap = 'jet')
        
        return adata_sample 
    
    else: 
        
        print('Building uterovaginal axis for samples {} and {} from {} (weight: {}) to {} (weight: {})'.format(sample1, sample2, start_feature, 
                            start_weight, end_feature, end_weight))
        
        # Read stitched sample directly 
        adata_stitched = sc.read(path_to_reptract + sample1 + '_' + sample2 + '/' + sample1 + '_' + sample2 + '_annotated.h5ad')
        
        # Add a prefix to the distances computed on the joint object 
        for col in adata_stitched.obs.columns:
            if col in [ 'L2_dist_longitudinal_anatomy_annots_' + start_feature, 'L2_dist_longitudinal_anatomy_annots_' + end_feature, 'broad_anatomy_annots']:
                adata_stitched.obs.rename(columns={col : 'joint_' + col}, inplace = True)
                
        # Concatenate 
        adata_separate = select_slide(adata, sample1, batch_key = 'sample').concatenate(select_slide(adata, sample2, batch_key = 'sample'), index_unique = None)
        adata_separate = adata_separate[[i > min_cells for i in adata_separate.obs['tot_cell_abundance']]]
        
        # Import distances 
        for col in adata_stitched.obs.columns:
            if col.startswith('joint') or col == 'overlaps_with':
                adata_separate.obs[col] = adata_separate.obs_names.map(adata_stitched.obs[col].to_dict())
            
        # Create a dictionary mapping the index of the missing value to the index of the replacement value
        replacement_dict = adata_separate.obs['overlaps_with'].dropna().to_dict()
        replacement_dict = {v:k for k,v in replacement_dict.items()}
        
        to_remove = []
        for col in adata_separate.obs.columns:
            if col.startswith('joint'):
                for spot_id in adata_separate.obs_names:
                    value = adata_separate.obs.at[spot_id, col]
                    if isinstance(value, (float, np.float64)) and np.isnan(value):
                        if spot_id in list(replacement_dict.keys()):
                            overlapping_spot = replacement_dict[spot_id]
                            if overlapping_spot in adata_separate.obs_names:
                                adata_separate.obs.at[spot_id, col] = adata_separate.obs.at[overlapping_spot, col]
                        elif spot_id not in list(replacement_dict.keys()):
                            to_remove.append(spot_id)
        remove = list(np.unique(to_remove))
        
        # create a boolean mask of the observations to keep
        mask = ~adata_separate.obs_names.isin(remove)

        # select only the rows that match the mask
        adata_separate = adata_separate[mask]
        
        adata_separate = adata_separate[[i not in excluded_regions for i in adata_separate.obs['broad_anatomy_annots']]]
        
        df = adata_separate.obs[['joint_L2_dist_longitudinal_anatomy_' + start_feature, 'joint_L2_dist_longitudinal_anatomy_' + end_feature, 
               'joint_broad_anatomy_annots']]
        fallopian_axis = [( start_weight*int(s) - end_weight*int(e) ) / ( start_weight*int(s) + end_weight*int(e) ) for s,e in zip(df['joint_L2_dist_longitudinal_anatomy_' + start_feature], df['joint_L2_dist_longitudinal_anatomy_' + end_feature])]
        df['Fallopian_Axis'] = fallopian_axis
        
        print(df['Fallopian_Axis'].hist(bins=20))
        
        # Add axis to slide 
        adata_separate.obs['Fallopian_Axis'] = adata_separate.obs_names.map(df['Fallopian_Axis'].to_dict())
        return adata_separate
        


In [ ]:
adata_cytassist.obs['longitudinal_anatomy'].value_counts()

In [ ]:
HCA_F_RepTsp13902018_axis = build_fallopian_axis(adata_cytassist, 'HCA_F_RepTsp13902018', 10, 'FallopianFimbriae', 'FallopianUterineJunction', 
                                                                      start_weight = 0.5, end_weight = 0.5, is_stitched = False)

In [ ]:
fallopian_HCA_F_RepTsp13902018.obs_names = ['HCA_F_RepTsp13902018_' + i for i in fallopian_HCA_F_RepTsp13902018.obs_names.to_list()]

In [ ]:
fallopian_HCA_F_RepTsp13902018.obs['FallopianAxis'] = fallopian_HCA_F_RepTsp13902018.obs_names.map(HCA_F_RepTsp13902018_axis.obs['Fallopian_Axis'].to_dict())

In [ ]:
from matplotlib.colors import LinearSegmentedColormap


# Define the colors and their corresponding positions
colors = ["#5e3c99", "#b2abd2", "#f1a1c5"]

# Create the colormap
cmap_fallopian = LinearSegmentedColormap.from_list('custom_cmap', colors, N=256)

In [ ]:
sc.pl.spatial(fallopian_HCA_F_RepTsp13902018, color = 'FallopianAxis', cmap = cmap_fallopian, bw = True,
              alpha_img = 0.7, spot_size = 50,
             save = 'HCA_F_RepTsp13902018_fallopian_axis_newcmap')

In [ ]:
adata_cytassist.obs['sample'].value_counts()

In [ ]:
HCA_F_RepTsp14645816_axis = build_fallopian_axis(adata_cytassist, 'HCA_F_RepTsp14645816', 10, 'FallopianFimbriae', 'FallopianUterineJunction', 
                                                                      start_weight = 0.5, end_weight = 0.5, is_stitched = False)

In [ ]:
for sample in [HCA_F_RepTsp13902018_axis, HCA_F_RepTsp14645816_axis]:
    sample.obs['FemaleReproductiveAxis'] = sample.obs['Fallopian_Axis']

In [ ]:
fallopian_axis = combine_axis(samples = [
                                           HCA_F_RepTsp13902018_axis,HCA_F_RepTsp14645816_axis
                                           ], 
                                axis_name = 'FemaleReproductiveAxis', return_adata = True)

In [ ]:
fallopian_axis

In [ ]:
fallopian_axis.obs.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/FallopianAxis_spots_cytassist.csv')

In [ ]:
fallopian_axis.var.head()

In [ ]:
fallopian_axis

In [ ]:
# Selecting relevant columns for computation
relevant_columns = [ 'Coelomic_Epithelium', 'FallopianTube_Epithelium', 'Ciliated_Epithelium', 'FallopianTube_Mesenchyme', 'FallopianTube_Ligament', 'FallopianTube_SmoothMuscle', 'Uterus/Cervix_Epithelium', 'Uterus_Mesenchyme', 'Uterus_Ligament', 'Uterus_SmoothMuscle', 'Cervix_Mesenchyme', 'Cervix/MüllerianVagina_Ligament', 'Cervix/MüllerianVagina_SmoothMuscle', 'MüllerianVagina_Epithelium', 'MüllerianVagina_Mesenchyme', 'VaginalPlate_Epithelium', 'VaginalPlate_Mesenchyme', 'VaginalPlate_SmoothMuscle', 'VaginalPlate_Ligament', 'MesonephricTubules_Epithelium', 'Epoophron_Mesenchyme', 'Pre-Perivascular', 'Perivascular', 'SkeletalMuscle', 'Endothelial', 'Endothelial_Lymphatic', 'Immune', 'Erythroid', 'SchwannCell', 'Neural']

# Calculating the percentage of cells that are of the most abundant cell type for each spot
fallopian_axis.obs['Most_Abundant_Cell_Type'] = fallopian_axis.obs[relevant_columns].idxmax(axis=1)
fallopian_axis.obs['Most_Abundant_Cell_Type'].value_counts()

In [ ]:
for s in fallopian_axis.obs_names:
    ct = fallopian_axis.obs.loc[s, 'Most_Abundant_Cell_Type']
    n_ct = fallopian_axis.obs.loc[s, ct]
    n_tot = fallopian_axis.obs.loc[s, relevant_columns].sum()
    fallopian_axis.obs.loc[s, 'Percentage_Most_Abundant_Cell_Type'] = (n_ct / n_tot) 

## 10. Fallopian axis in epithelial spots -- prepare for TradeSeq

In [ ]:
fallopian_epi = fallopian_axis[[i == 'Epithelium' for i in fallopian_axis.obs['histology_annots']]]

In [ ]:
fallopian_epi = fallopian_epi[[i in ['FallopianTube_Epithelium', 'Uterus/Cervix_Epithelium', 
                                     ] for i in fallopian_epi.obs['Most_Abundant_Cell_Type']]]
fallopian_epi

In [ ]:
fallopian_epi.obs['Percentage_Most_Abundant_Cell_Type'].hist(bins = 20)

In [ ]:
# fallopian_epi = fallopian_epi[[i > 0.4 for i in fallopian_epi.obs['Percentage_Most_Abundant_Cell_Type']]]

In [ ]:


HCA_F_RepTsp14645816_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14645816_axis.obs_names.map(fallopian_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14645816_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14645816_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14645816_axis.obs_names.map(fallopian_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14645816_axis, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp13902018_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902018_axis.obs_names.map(fallopian_epi.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902018_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902018_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902018_axis.obs_names.map(fallopian_epi.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902018_axis, color="Most_Abundant_Cell_Type")



In [ ]:
fallopian_epi.X[20:30, 20:30].toarray()

In [ ]:
fallopian_epi_forR = fallopian_epi.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in fallopian_epi_forR.var_names if not name.startswith('MT-')]
fallopian_epi_forR = fallopian_epi_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in fallopian_epi_forR.var_names if not name.startswith('RP')]
fallopian_epi_forR = fallopian_epi_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in fallopian_epi_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
fallopian_epi_forR = fallopian_epi_forR[:, non_haemo_genes]

sc.pp.filter_genes(fallopian_epi_forR, min_cells = 10)
sc.pp.filter_genes(fallopian_epi_forR, min_counts = 1)

In [ ]:
fallopian_epi_forR.shape

In [ ]:
for col in fallopian_epi_forR.obs.columns:
    if col not in [ 'FemaleReproductiveAxis', 'sample']:
        del fallopian_epi_forR.obs[col]

In [ ]:
fallopian_epi_forR.obs['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
del fallopian_epi_forR.obsm

In [ ]:
%%R -i fallopian_epi_forR
fallopian_epi_forR

In [ ]:
%%R 
saveRDS(fallopian_epi_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/fallopianaxis_epi_cytassist.rds')

## 11. Fallopian axis in mesenchymal spots -- prepare for TradeSeq

In [ ]:
fallopian_axis.obs['Most_Abundant_Cell_Type'].value_counts()

In [ ]:
fallopian_mese = fallopian_axis[[i in ['FallopianTube_Mesenchyme', 
                                     ] for i in fallopian_axis.obs['Most_Abundant_Cell_Type']]]
fallopian_mese

In [ ]:
fallopian_mese.obs['Percentage_Most_Abundant_Cell_Type'].hist(bins = 50)

In [ ]:
fallopian_mese = fallopian_mese[[i > 0.5 for i in fallopian_mese.obs['Percentage_Most_Abundant_Cell_Type']]]

In [ ]:
fallopian_mese.shape

In [ ]:
fallopian_mese.obs['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
HCA_F_RepTsp14645816_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp14645816_axis.obs_names.map(fallopian_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14645816_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp14645816_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp14645816_axis.obs_names.map(fallopian_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp14645816_axis, color="Most_Abundant_Cell_Type")

HCA_F_RepTsp13902018_axis.obs['Percentage_Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902018_axis.obs_names.map(fallopian_mese.obs['Percentage_Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902018_axis, color="Percentage_Most_Abundant_Cell_Type")
HCA_F_RepTsp13902018_axis.obs['Most_Abundant_Cell_Type'] = HCA_F_RepTsp13902018_axis.obs_names.map(fallopian_mese.obs['Most_Abundant_Cell_Type'].to_dict())
sc.pl.spatial(HCA_F_RepTsp13902018_axis, color="Most_Abundant_Cell_Type")



In [ ]:
fallopian_mese_forR = fallopian_mese.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in fallopian_mese_forR.var_names if not name.startswith('MT-')]
fallopian_mese_forR = fallopian_mese_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in fallopian_mese_forR.var_names if not name.startswith('RP')]
fallopian_mese_forR = fallopian_mese_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in fallopian_mese_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
fallopian_mese_forR = fallopian_mese_forR[:, non_haemo_genes]

sc.pp.filter_genes(fallopian_epi_forR, min_cells = 10)
sc.pp.filter_genes(fallopian_epi_forR, min_counts = 1)

In [ ]:
for col in fallopian_mese_forR.obs.columns:
    if col not in [ 'FemaleReproductiveAxis', 'sample']:
        del fallopian_mese_forR.obs[col]

In [ ]:
del fallopian_mese_forR.obsm

In [ ]:
%%R -i fallopian_mese_forR
fallopian_mese_forR

In [ ]:
%%R 
saveRDS(fallopian_mese_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/fallopianaxis_mese_cytassist.rds')

## 12. Append the fallopian and uterovaginal axis to obtain the Müllerian longitudinal axis

### Transpose the values of the fallopian axis to append them to the uterovaginal axis

In [ ]:
fallopian_axis = fallopian_axis.obs.copy()

In [ ]:
fallopian_axis['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
uterovaginal_axis['FemaleReproductiveAxis'] = uterovaginal_axis['UteroVaginal_Axis_forPlotting'].copy()

In [ ]:
uterovaginal_axis['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
axis = fallopian_axis.append(uterovaginal_axis)

In [ ]:
axis['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
axis.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/FemaleReproductiveAxis_spots_cytassist.csv')

In [ ]:
axis['sample'].value_counts()

## Bin the axis and downsample spots to have approximately same number of spots per bin

In [ ]:
def bin_axis(df, n_bins = 10, axis_name = 'FemaleReproductiveAxis'):
    max_val = np.nanquantile(df[axis_name],0.9999)
    min_val = np.nanquantile(df[axis_name],0.0001)

    incr_uva = (max_val-min_val)/n_bins
    uva_bins = []
    for i in range(0,n_bins+1):
        uva_bins.append(min_val+(i*incr_uva))

    df['binned_' + axis_name + '_' + str(n_bins) + '_bins'] = pd.cut(df[axis_name], bins = uva_bins, labels=uva_bins[:-1])
    return df


In [ ]:
# Function to downsample spots
def downsample_spots(group):
    max_spots = 1000
    if len(group) > max_spots:
        return group.sample(max_spots)
    else:
        return group

In [ ]:
for s in [HCA_F_RepTsp13902013, HCA_F_RepTsp13902014, HCA_F_RepTsp13902015, HCA_F_RepTsp13902016]:
    s.obs['FemaleReproductiveAxis'] = s.obs_names.map(s.obs['UteroVaginal_Axis_forPlotting'])

In [ ]:
axis_adata = combine_axis(samples = [
                                            HCA_F_RepTsp13902018_axis, HCA_F_RepTsp14645816_axis, 
    HCA_F_RepTsp13902013, HCA_F_RepTsp13902014, HCA_F_RepTsp13902015, HCA_F_RepTsp13902016
                                           ], 
                                axis_name = 'FemaleReproductiveAxis', return_adata = True)

In [ ]:
axis_adata.obs['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
# axis_adata = axis_adata[[i for i in axis_adata.obs_names.to_list() if i in downsampled_axis_cleaned.index.to_list()], :]
axis_adata.shape

In [ ]:
axis['FemaleReproductiveAxis']

In [ ]:
axis_adata.obs['FemaleReproductiveAxis'] = axis_adata.obs_names.map(axis['FemaleReproductiveAxis'].to_dict())

In [ ]:
axis_adata.obs['FemaleReproductiveAxis'].hist(bins = 20)

In [ ]:
fallopian_mese_spots = fallopian_mese.obs_names.to_list()
fallopian_epi_spots = fallopian_epi.obs_names.to_list()

In [ ]:
uterus_epi_spots = uterus_epi.obs_names.to_list()
uterus_mese_spots = uterus_mese.obs_names.to_list()

In [ ]:
len(fallopian_mese_spots)+ len(uterus_mese_spots)

In [ ]:
fallopian_mese_spots.extend(uterus_mese_spots)

In [ ]:
len(fallopian_mese_spots)

In [ ]:
len(fallopian_epi_spots) + len(uterus_epi_spots)

In [ ]:
fallopian_epi_spots.extend(uterus_epi_spots)

In [ ]:
len(fallopian_epi_spots)

In [ ]:
axis_adata_mese = axis_adata[[i for i in axis_adata.obs_names.to_list() if i in fallopian_mese_spots], :]

In [ ]:
axis_adata_mese

In [ ]:
axis_adata_mese.obs['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
def bin_axis(df, n_bins = 10, axis_name = 'FemaleReproductiveAxis'):
    max_val = np.nanquantile(df[axis_name],0.9999)
    min_val = np.nanquantile(df[axis_name],0.0001)

    incr_uva = (max_val-min_val)/n_bins
    uva_bins = []
    for i in range(0,n_bins+1):
        uva_bins.append(min_val+(i*incr_uva))

    df['binned_' + axis_name + '_' + str(n_bins) + '_bins'] = pd.cut(df[axis_name], bins = uva_bins, labels=uva_bins[:-1])
    return df

In [ ]:
axis = bin_axis(axis_adata_mese.obs, n_bins = 10, axis_name = 'FemaleReproductiveAxis')

In [ ]:
tfs_to_plot = ['PROX1', 'GATA6', 'NFATC2','LEF1',
               'FOXL2',    'MEIS2','EMX2', 'FOXO1', 'ESR1', 'RORB','HMGA2','MSX1',  'AR',
               'TWIST1',
               'ESRRG', 'RUNX1','PRRX2','TWIST2',  'LBX2', 'PBX3',
              'AHR',  'EVX1', 'EVX2', 'IRF6','NR0B1', 'ISL1', 'HMBOX1',   
 'ASCL2', 'TBX18'
              ]

In [ ]:
axis_adata_mese

In [ ]:
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=150, frameon=True, vector_friendly=True, 
                         fontsize=14, figsize=(7, 7), color_map=None, format='pdf', facecolor=None, transparent=False)

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
# Create matrix plot using scanpy
sc.pl.matrixplot(axis_adata_mese,var_names=tfs_to_plot,groupby='binned_FemaleReproductiveAxis_10_bins',
                 num_categories = 10,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,
                 save='visium_matrixplot_mese_tfs_spatiallyvariable.pdf'
                )

In [ ]:
ligands_to_plot = ['LGR5', 'CD36', 'NTRK2', 'CD55', 'ALDH1A2', 'SFRP4', 'DLK1','NRG1', 'FGF7',
            'WNT4',  'NRXN3',  'FLRT2', 'WNT5A', 'GRIA4',  'TNFRSF21','TGM2', 'ALDH1A1', 'LRRTM1', 'IGFBP3', 'NRP2','RORB',
            'COL26A1','ROBO2',  'GDF7','AR', 'NRXN1', 'WIF1', 'WNT11',  'SCARA5','SFRP5', 'EPHA5', 
           'IGF1', 'BMP4', 'BMP7', ]

In [ ]:
# Create matrix plot using scanpy
sc.pl.matrixplot(axis_adata_mese,var_names=ligands_to_plot,groupby='binned_FemaleReproductiveAxis_10_bins',
                 num_categories = 10,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,
                 save='visium_matrixplot_mese_ligands_selected_spatiallyvariable.pdf'
                )

In [ ]:
# Random downsampling per cell type 
import random
from itertools import chain
def downsample(adata, labels, n): 
    
    myindex = adata.obs[labels].value_counts().index 
    myvalues = adata.obs[labels].value_counts().values
    clusters = pd.Series(myvalues, index = myindex)
    
    # Find clusters with > n cells 
    cl2downsample = clusters.index[ clusters.values > n ]

    # save all barcode ids from small clusters
    holder = []
    holder.append( adata.obs_names[[ i not in cl2downsample for i in adata.obs[labels] ]] ) 

    # randomly sample n cells in the cl2downsample
    for cl in cl2downsample:
        print(cl)
        cl_sample = adata[[ i == cl for i in adata.obs[labels]]].obs_names
        
        cl_downsample = random.sample(set(cl_sample), n )
        holder.append(cl_downsample)
    
    # samples to include
    samples = list(chain(*holder))

    # Filter adata_count
    adata = adata[[ i in samples for i in adata.obs_names ]]
    return adata

In [ ]:
axis_adata_mese = downsample(axis_adata_mese, 'binned_FemaleReproductiveAxis_10_bins', 50)

In [ ]:
axis_adata_mese.obs['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
axis_adata_mese.X[20:30, 20:30].toarray()

In [ ]:
axis_adata_mese.shape

In [ ]:
axis_adata_mese_forR = axis_adata_mese.copy()

In [ ]:
# Create matrix plot using scanpy
sc.pl.matrixplot(axis_adata_mese,var_names=ligands_to_plot,groupby='binned_FemaleReproductiveAxis_10_bins',
                 num_categories = 10,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,
                 save='visium_matrixplot_mese_ligands_selected_spatiallyvariable_downsampled.pdf'
                )

In [ ]:
mito_genes = [name for name in axis_adata_mese_forR.var_names if name.startswith('MT')]

In [ ]:
# remove mito genes
non_mito_genes = [name for name in axis_adata_mese_forR.var_names if not name.startswith('MT')]
axis_adata_mese_forR = axis_adata_mese_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in axis_adata_mese_forR.var_names if not name.startswith('RP')]
axis_adata_mese_forR = axis_adata_mese_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in axis_adata_mese_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
axis_adata_mese_forR = axis_adata_mese_forR[:, non_haemo_genes]


# remove heat shock protein genes
non_hps_genes = [name for name in axis_adata_mese_forR.var_names if not name.startswith('HSP')]
axis_adata_mese_forR = axis_adata_mese_forR[:, non_hps_genes]

sc.pp.filter_genes(axis_adata_mese_forR, min_cells = 10)
sc.pp.filter_genes(axis_adata_mese_forR, min_counts = 1)

In [ ]:
axis_adata_mese_forR.shape

In [ ]:
for col in axis_adata_mese_forR.obs.columns:
    if col not in [ 'FemaleReproductiveAxis', 'sample']:
        del axis_adata_mese_forR.obs[col]

In [ ]:
del axis_adata_mese_forR.obsm

In [ ]:
%%R -i axis_adata_mese_forR
axis_adata_mese_forR

In [ ]:
%%R 
saveRDS(axis_adata_mese_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/femalereproductiveaxis_mese_cytassist_downsampled.rds')

In [ ]:
axis_adata_mese_forR

In [ ]:
axis_adata_mese_forR.obs.to_csv('/lustre/scratch126/cellgen/team292/vl6/VISIUM/femalereproductiveaxis_mese_spots_cytassist.csv')

In [ ]:
axis_adata.shape

In [ ]:
len(fallopian_epi_spots)

In [ ]:
axis_adata_epi = axis_adata[[i for i in axis_adata.obs_names.to_list() if i in fallopian_epi_spots], :]

In [ ]:
axis_adata_epi.obs['FemaleReproductiveAxis'].hist(bins = 10)

In [ ]:
axis = bin_axis(axis_adata_epi.obs, n_bins = 10, axis_name = 'FemaleReproductiveAxis')

In [ ]:
axis

In [ ]:
axis['binned_FemaleReproductiveAxis_10_bins'] = axis['binned_FemaleReproductiveAxis_10_bins'].astype(float)

In [ ]:
axis['binned_FemaleReproductiveAxis_10_bins'] = np.round(axis['binned_FemaleReproductiveAxis_10_bins'], 2)

In [ ]:
axis_adata_epi.obs['binned_FemaleReproductiveAxis_10_bins'] = axis_adata_epi.obs_names.map(axis['binned_FemaleReproductiveAxis_10_bins'].to_dict())

In [ ]:
axis_adata_epi.obs['binned_FemaleReproductiveAxis_10_bins'].value_counts()

In [ ]:
axis_adata_epi = downsample(axis_adata_epi, 'binned_FemaleReproductiveAxis_10_bins', 50)

In [ ]:
axis_adata_epi.obs['binned_FemaleReproductiveAxis_10_bins'].value_counts()

In [ ]:
axis_adata_epi.X[20:30, 20:30].toarray()

In [ ]:
axis_adata_epi_forR = axis_adata_epi.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in axis_adata_epi_forR.var_names if not name.startswith('MT')]
axis_adata_epi_forR = axis_adata_epi_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in axis_adata_epi_forR.var_names if not name.startswith('RP')]
axis_adata_epi_forR = axis_adata_epi_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in axis_adata_epi_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
axis_adata_epi_forR = axis_adata_epi_forR[:, non_haemo_genes]


# remove heat shock protein genes
non_hps_genes = [name for name in axis_adata_epi_forR.var_names if not name.startswith('HSP')]
axis_adata_epi_forR = axis_adata_epi_forR[:, non_hps_genes]

sc.pp.filter_genes(axis_adata_epi_forR, min_cells = 10)
sc.pp.filter_genes(axis_adata_epi_forR, min_counts = 1)

In [ ]:
axis_adata_epi_forR.shape

In [ ]:
for col in axis_adata_epi_forR.obs.columns:
    if col not in [ 'FemaleReproductiveAxis', 'sample']:
        del axis_adata_epi_forR.obs[col]

In [ ]:
del axis_adata_epi_forR.obsm

In [ ]:
%%R -i axis_adata_epi_forR
axis_adata_epi_forR

In [ ]:
%%R 
saveRDS(axis_adata_epi_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/femalereproductiveaxis_epi_cytassist_downsampled.rds')

In [ ]:
axis_adata_epi

## Epithelial spots of Fallopian axis only for intra-organ differences 

In [ ]:
axis_adata_epi_FT = axis_adata_epi[[i < 0 for i in axis_adata_epi.obs['FemaleReproductiveAxis']]]
axis_adata_epi_FT.shape

In [ ]:
axis = bin_axis(axis_adata_epi_FT.obs, n_bins = 3, axis_name = 'FemaleReproductiveAxis')
axis['binned_FemaleReproductiveAxis_3_bins'] = axis['binned_FemaleReproductiveAxis_3_bins'].astype(float)
axis['binned_FemaleReproductiveAxis_3_bins'] = np.round(axis['binned_FemaleReproductiveAxis_3_bins'], 2)
axis_adata_epi_FT.obs['binned_FemaleReproductiveAxis_3_bins'] = axis_adata_epi_FT.obs_names.map(axis['binned_FemaleReproductiveAxis_3_bins'].to_dict())

In [ ]:
axis_adata_epi_FT.obs['binned_FemaleReproductiveAxis_3_bins'].value_counts()

In [ ]:
axis_adata_epi_FT = downsample(axis_adata_epi_FT, 'binned_FemaleReproductiveAxis_3_bins', 30)

In [ ]:
axis_adata_epi_FT.X[20:30, 20:30].toarray()

In [ ]:
axis_adata_epi_FT.obs['FemaleReproductiveAxis'].hist(bins = 3)

In [ ]:
axis_adata_epi_FT_forR = axis_adata_epi_FT.copy()

In [ ]:
# remove mito genes
non_mito_genes = [name for name in axis_adata_epi_FT_forR.var_names if not name.startswith('MT')]
axis_adata_epi_FT_forR = axis_adata_epi_FT_forR[:, non_mito_genes]

# remove ribo genes
non_ribo_genes = [name for name in axis_adata_epi_FT_forR.var_names if not name.startswith('RP')]
axis_adata_epi_FT_forR = axis_adata_epi_FT_forR[:, non_ribo_genes]

# remove haemoglobin genes 
non_haemo_genes = [name for name in axis_adata_epi_FT_forR.var_names if name not in ['HBA1', 'HBA2', 'HBB', 'HBG1', 'HBG2']]
axis_adata_epi_FT_forR = axis_adata_epi_FT_forR[:, non_haemo_genes]


# remove heat shock protein genes
non_hps_genes = [name for name in axis_adata_epi_FT_forR.var_names if not name.startswith('HSP')]
axis_adata_epi_FT_forR = axis_adata_epi_FT_forR[:, non_hps_genes]

sc.pp.filter_genes(axis_adata_epi_FT_forR, min_cells = 10)
sc.pp.filter_genes(axis_adata_epi_FT_forR, min_counts = 1)

In [ ]:
axis_adata_epi_FT_forR.shape

In [ ]:
for col in axis_adata_epi_FT_forR.obs.columns:
    if col not in [ 'FemaleReproductiveAxis', 'sample']:
        del axis_adata_epi_FT_forR.obs[col]

In [ ]:
del axis_adata_epi_FT_forR.obsm

In [ ]:
%%R -i axis_adata_epi_FT_forR
axis_adata_epi_FT_forR

In [ ]:
%%R 
saveRDS(axis_adata_epi_FT_forR, '/lustre/scratch126/cellgen/team292/vl6/VISIUM/femalereproductiveaxis_epi_cytassist_downsampled_fallopiantube.rds')

In [ ]:
variable_signature = [ 
     'PPP2R2B','GATA6','APOA1','PNOC','MMP28', 'DLGAP1',  'RSPO1',  'ERP27',  'WT1',  'ALDH1A2', 
    'CD109', 'DOK5','LYPD1',  'TTYH1',  'CRTAC1',
        
       'EME2', 'ETV4', 'HAPLN3', 'SP5','MUC6', 'KCNN4',  'CHRNA4',  'SHANK1', 'FGF17', 'FGFR3', 'WDR72',
        
             ]

In [ ]:
axis_adata_epi_FT.raw = axis_adata_epi_FT.copy()
sc.pp.normalize_total(axis_adata_epi_FT, inplace=True)
sc.pp.log1p(axis_adata_epi_FT)

In [ ]:
# Create matrix plot using scanpy
sc.pl.matrixplot(axis_adata_epi_FT,var_names=variable_signature,groupby='binned_FemaleReproductiveAxis_3_bins',
                 num_categories = 3,
                 cmap='OrRd',
                 standard_scale='var',dendrogram=False,save='matrixplot_epi_fallopian_spatiallyvariable.pdf')